In [1]:
import sys 
sys.path.append("./script")

from data_preparation import DashDataPreparation
from Logit_utils import GridScore, LogitModel
from XGB_utils import XGB_model, GridScoreXGB

# Préparation des données

In [2]:
dataprep = DashDataPreparation()

In [3]:
dataprep.intialize_data()

Variables extérieures récupérées ✅
Type des variables convertis ✅
Valeurs manquantes traitées ✅


In [4]:
dataprep.init_vars( ['REGION_RATING_CLIENT_W_CITY', 'DAYS_CREDIT_ENDDATE', 'RATE_DOWN_PAYMENT', 'AMT_PAYMENT', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE'])

In [5]:
train_prepared = dataprep.get_prepared_data()

Discrétisation des variables numériques en cours ... 


100%|██████████| 3/3 [00:58<00:00, 19.56s/it]


Variables numériques discrétisées ✅
Discrétisation des variables catégorielles en cours ... 
Variables catégorielles discrétisées ✅


# Modèle XGB

In [6]:
model = XGB_model(train_prepared, dataprep.discretizer.intervalles_dic)
shap_df = model.run_xgb_model()

In [7]:
gs = GridScoreXGB(train_prepared, shap_df)
grid_score = gs.compute_grid_score()
grid_score

,Variable,Modality,Coefficient,Score,Contribution,Pcentage_Défaut,Pcentage_Classe
0,RATE_DOWN_PAYMENT_disc_int,[0.2;4.48]_ref,-0.173039,112.870810,11.0,2.03,32.47
1,RATE_DOWN_PAYMENT_disc_int,[-0.0;0.2],0.073328,0.000000,11.0,5.64,67.53
2,AMT_PAYMENT_disc_int,[123720.66;25537053.78]_ref,-0.057911,90.103632,9.0,5.17,72.52
3,AMT_PAYMENT_disc_int,[0.0;123720.16],0.138762,0.000000,9.0,2.50,27.48
4,REGION_RATING_CLIENT_W_CITY,un_deux_ref,-0.071786,199.629595,20.0,6.13,85.76
5,REGION_RATING_CLIENT_W_CITY,trois,0.363954,0.000000,20.0,1.55,14.24
6,OCCUPATION_TYPE_discret,high_skilled_ref,-0.285343,165.400836,17.0,1.35,23.12
7,OCCUPATION_TYPE_discret,low_skilled,0.075684,0.000000,17.0,6.32,76.88
8,NAME_INCOME_TYPE_discret,Low_income_ref,-0.382180,222.582780,22.0,1.31,25.11
9,NAME_INCOME_TYPE_discret,high_income,0.103660,0.000000,22.0,6.36,74.89


In [8]:
model.compute_metrics()

{'roc_auc': 0.6177352120548587,
 'gini': 0.2354704241097174,
 'fpr': array([0.        , 0.00870411, 0.02782839, 0.0298452 , 0.03596639,
        0.09953118, 0.10282176, 0.11154356, 0.11476338, 0.1216276 ,
        0.24352057, 0.25797435, 0.2594958 , 0.26007961, 0.27538257,
        0.28387439, 0.2848651 , 0.28534277, 0.32544892, 0.32728881,
        0.329341  , 0.33286157, 0.35469261, 0.42331712, 0.43302963,
        0.45233083, 0.45250774, 0.45730208, 0.49997346, 0.50367094,
        0.50687307, 0.5076161 , 0.52053074, 0.52605042, 0.61427687,
        0.65882353, 0.65981424, 0.66430783, 0.66828837, 0.66906678,
        0.67840778, 0.6793985 , 0.68320212, 0.69772667, 0.72242371,
        0.7407165 , 0.80769571, 0.80946484, 0.81797435, 0.81935427,
        0.8377709 , 0.83789474, 0.84229987, 0.87302963, 0.87686864,
        0.88260062, 0.88702344, 0.88806723, 0.93551526, 0.93838125,
        0.96212295, 0.98715613, 0.98798762, 0.99501106, 1.        ]),
 'tpr': array([0.        , 0.02618693, 0.06855

# Modèle Logit

In [9]:
model = LogitModel(train_prepared, dataprep.discretizer.intervalles_dic)

In [10]:
logit = model.run_model()

Optimization terminated successfully.
         Current function value: 0.261461
         Iterations 7


In [11]:
GS = GridScore(train_prepared, logit)
grid_score = GS.compute_grid_score()
grid_score

,Variable,Modality,Coefficient,P-Value,Score,Contribution,Pcentage_Défaut,Pcentage_Classe
0,Intercept,-,-3.865127,0.0,0.000000,0.0,0.00,0.00
1,DAYS_CREDIT_ENDDATE_disc_int,[-155271.0;-207.0]_ref,0.000000,0.0,195.744681,20.0,2.60,45.01
2,DAYS_CREDIT_ENDDATE_disc_int,[-206.0;214193.0],0.460000,0.0,0.000000,20.0,5.07,54.99
3,RATE_DOWN_PAYMENT_disc_int,[0.2;4.48]_ref,0.000000,0.0,102.127660,10.0,2.03,32.47
4,RATE_DOWN_PAYMENT_disc_int,[-0.0;0.2],0.240000,0.0,0.000000,10.0,5.64,67.53
5,AMT_PAYMENT_disc_int,[123720.66;25537053.78]_ref,0.000000,0.0,85.106383,9.0,5.17,72.52
6,AMT_PAYMENT_disc_int,[0.0;123720.16],0.200000,0.0,0.000000,9.0,2.50,27.48
7,NAME_INCOME_TYPE_discret,Low_income_ref,0.000000,0.0,234.042553,23.0,1.31,25.11
8,NAME_INCOME_TYPE_discret,high_income,0.550000,0.0,0.000000,23.0,6.36,74.89
9,OCCUPATION_TYPE_discret,high_skilled_ref,0.000000,0.0,182.978723,18.0,1.35,23.12


In [13]:
model.get_metrics()

{'fpr': array([0.        , 0.00975282, 0.02904826, 0.03136035, 0.03560619,
        0.04552716, 0.05170674, 0.11484782, 0.11753825, 0.12506306,
        0.14007062, 0.25971078, 0.26034135, 0.26185472, 0.2785438 ,
        0.28682529, 0.28716159, 0.29018833, 0.29229023, 0.31457037,
        0.32428115, 0.39171011, 0.43055322, 0.43156213, 0.43479906,
        0.43807802, 0.45800404, 0.46317471, 0.5042038 , 0.59420716,
        0.59492181, 0.60076509, 0.60328737, 0.61514209, 0.62443249,
        0.66932907, 0.66937111, 0.6701278 , 0.67407937, 0.67731629,
        0.70144611, 0.71607533, 0.78346225, 0.78442912, 0.78501766,
        0.78896923, 0.79296284, 0.81141752, 0.82886329, 0.85879435,
        0.86320834, 0.86467967, 0.87418026, 0.92138894, 0.92147301,
        0.92269211, 0.92723222, 0.95186649, 0.95295948, 0.95808811,
        0.96065243, 0.98680007, 0.99365226, 0.9945771 , 1.        ]),
 'tpr': array([0.        , 0.02716688, 0.07546356, 0.08020699, 0.08710651,
        0.09874946, 0.11082363, 